In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#いろいろインポート
import pandas_profiling as pdp # 素敵な分析ツール

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random
from scipy import stats
import os
from IPython.core.display import HTML

In [ ]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [ ]:
train.info()
test.info()

- PassengerId-各乗客の一意のID。各IDはgggg_ppの形式を取ります。
ここで、ggggは乗客が一緒に旅行しているグループを示し、ppはグループ内のグループの番号です。
グループ内の人々はしばしば家族の一員ですが、常にではありません。
- HomePlanet-乗客が出発した惑星、通常は永住権の惑星。
- CryoSleep-航海中に乗客が中断されたアニメーションに入れられることを選択したかどうかを示します。
          クライオスリープの乗客はキャビンに閉じ込められます。
- キャビン-乗客が滞在しているキャビン番号。デッキ/番号/サイドの形式を取ります。サイドは、左舷の場合はP、右舷の場合はSのいずれかになります。
- 目的地-乗客が吠える惑星。
- 年齢-乗客の年齢。
- VIP-乗客が航海中に特別なVIPサービスの料金を支払ったかどうか。
- RoomService、FoodCourt、ShoppingMall、Spa、VRDeck-SpaceshipTitanicの多くの豪華なアメニティのそれぞれで乗客が請求した金額。
- 名前-乗客の名前と名前。
- 輸送-乗客が別の次元に輸送されたかどうか。これがターゲットであり、予測しようとしている列です

In [ ]:
# Changing bool values to 1 and 0 for easier visualisation: 

#train = train.replace({True: 1, False: 0})
#test = test.replace({True: 1, False: 0})

↓のレポートから得られる気づき的な　★はあとでやりたい
- Cryosleepは相関強そう。コールドスリープかな？
- AGE　年齢は意外と相関ない。
- Vipは生き残り多そうかと思ったら、相関は低い。のは、Vipperの母数が少ないからかも。
- ルームサービスとか、Spaとか、VRDeckとかにお金払っている人が生きのこっていそう。やはりお金持ち＝正義か。あと、お金項目は分散が強そう。指数にした方がいいかも？
- ★キャビンは右舷、左舷とかで影響ありそう。あとで分解してみよう。
- ★母星とか目的地とかはカテゴリ項目なので、相関取れない。ので、カテゴリごとの生死比率を出してみよう。
- ★Name　・・・　同じ姓だったら家族とか？使い道はありそう。

In [ ]:
# Split cabin into deck, number and side.
train['CabinDeck'] = train['Cabin'].apply(lambda x: x.split('/')[0] if isinstance(x, str) else x)
train['CabinNumber'] = train['Cabin'].apply(lambda x: int(x.split('/')[1]) if isinstance(x, str) else x)
train['CabinSide'] = train['Cabin'].apply(lambda x: x.split('/')[2] if isinstance(x, str) else x)

test['CabinDeck'] = test['Cabin'].apply(lambda x: x.split('/')[0] if isinstance(x, str) else x)
test['CabinNumber'] = test['Cabin'].apply(lambda x: int(x.split('/')[1]) if isinstance(x, str) else x)
test['CabinSide'] = test['Cabin'].apply(lambda x: x.split('/')[2] if isinstance(x, str) else x)
train[['Cabin', 'CabinDeck', 'CabinNumber', 'CabinSide']]

In [ ]:
'''
# stragegy = "most_frequent" が最頻値, "median" は中央値による補完
imputer = DataFrameMapper([
    (["HomePlanet"], [SimpleImputer(strategy="most_frequent")]),
    (["CryoSleep"], [SimpleImputer(strategy="most_frequent")])
    
#    (["VRDeck"], [SimpleImputer(strategy="median")])
    ], input_df=True, df_out=True, default=None
)
# 訓練データの学習
imputer.fit(train)
imputer.fit(test)
'''

In [ ]:
#RoomService, FoodCourt, ShoppingMall, Spa, VRDeck fillna 0

train['RoomService']=train['RoomService'].fillna(0)
train['FoodCourt']=train['FoodCourt'].fillna(0)
train['ShoppingMall']=train['ShoppingMall'].fillna(0)
train['Spa']=train['Spa'].fillna(0)
train['VRDeck']=train['VRDeck'].fillna(0)
train['VIP']=train['VIP'].fillna(0)

test['RoomService']=test['RoomService'].fillna(0)
test['FoodCourt']=test['FoodCourt'].fillna(0)
test['ShoppingMall']=test['ShoppingMall'].fillna(0)
test['Spa']=test['Spa'].fillna(0)
test['VRDeck']=test['VRDeck'].fillna(0)
test['VIP']=test['VIP'].fillna(0)

train['CryoSleep']=train['CryoSleep'].fillna(False)
test['CryoSleep']=test['CryoSleep'].fillna(False)

train['Cabin']=train['Cabin'].fillna(method='ffill')
test['Cabin']=test['Cabin'].fillna(method='ffill')

train['HomePlanet']=train['HomePlanet'].fillna('Earth')
test['HomePlanet']=test['HomePlanet'].fillna('Earth')

In [ ]:
# stragegy = "most_frequent" が最頻値, "median" は中央値による補完
imputer = DataFrameMapper([
    (["Destination"], [SimpleImputer(strategy="most_frequent")]),
    (["Age"], [SimpleImputer(strategy="median")])
    ], input_df=True, df_out=True, default=None
)

# 訓練データの学習
imputer.fit(train)
# 補完処理
train = imputer.transform(train)

# 訓練データの学習
imputer.fit(test)
# 補完処理
test = imputer.transform(test)

In [ ]:
train.info()

In [ ]:
#RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
train['ttl_spnd']=train['RoomService']+train['FoodCourt']+train['ShoppingMall']+train['Spa']+train['VRDeck']
test['ttl_spnd']=test['RoomService']+test['FoodCourt']+test['ShoppingMall']+test['Spa']+test['VRDeck']

In [ ]:
train['Adult']=True
train.loc[train['Age']<18, 'Adult']=False
test['Adult']=True
test.loc[test['Age']<18, 'Adult']=False

In [ ]:
# Split cabin into deck, number and side.
train['CabinDeck'] = train['Cabin'].apply(lambda x: x.split('/')[0] if isinstance(x, str) else x)
train['CabinNumber'] = train['Cabin'].apply(lambda x: int(x.split('/')[1]) if isinstance(x, str) else x)
train['CabinSide'] = train['Cabin'].apply(lambda x: x.split('/')[2] if isinstance(x, str) else x)

test['CabinDeck'] = test['Cabin'].apply(lambda x: x.split('/')[0] if isinstance(x, str) else x)
test['CabinNumber'] = test['Cabin'].apply(lambda x: int(x.split('/')[1]) if isinstance(x, str) else x)
test['CabinSide'] = test['Cabin'].apply(lambda x: x.split('/')[2] if isinstance(x, str) else x)
train[['Cabin', 'CabinDeck', 'CabinNumber', 'CabinSide']]

In [ ]:
# `PassangerGroup`: Passanger's group
# `PassengerNumber`: Unique within `PassengerGroup`
train['PassengerGroup'] = train['PassengerId'].apply(lambda x: str(x[:4]))
train['PassengerNumber'] = train['PassengerId'].apply(lambda x: int(x[-2:]))
test['PassengerGroup'] = test['PassengerId'].apply(lambda x: str(x[:4]))
test['PassengerNumber'] = test['PassengerId'].apply(lambda x: int(x[-2:]))
train[['PassengerId', 'PassengerGroup', 'PassengerNumber']]

In [ ]:
# `PassengerGroupSize`: How many people are in each `PassangerGroup`?
train['PassengerGroupSize'] = train.groupby('PassengerGroup')['PassengerNumber'].transform('max')
test['PassengerGroupSize'] = test.groupby('PassengerGroup')['PassengerNumber'].transform('max')
train[['PassengerGroup', 'PassengerNumber', 'PassengerGroupSize']].sort_values(['PassengerGroup', 'PassengerNumber'])

In [ ]:
categorical_features = [c for c in train.select_dtypes(exclude='number').columns if c not in ('Transported', 'Cabin', 'Name', 'PassengerId', 'PassengerGroup')]
categorical_features

In [ ]:
for c in categorical_features:
    values = [v for v in train[c].unique() if not (isinstance(v, float) and np.isnan(v))]
    mapping = {v: i for i, v in enumerate(values)}
    print(f'''{c};
    - Mapping: {mapping}
    - Values (training set): {train[c].value_counts().sort_index().index.tolist()}
    - Values (test set)    : {test[c].value_counts().sort_index().index.tolist()}''')
    train[c] = train[c].map(mapping)
    test[c] = test[c].map(mapping)

In [ ]:
train=train.drop(columns=['Cabin','Name'])
test=test.drop(columns=['Cabin','Name'])

In [ ]:
train.head()

In [ ]:
test

ここからモデル作成に入ります

In [ ]:
# scikit-learnのインポートをします
from sklearn import tree
#RandomForestClassifier

# 「train」の目的変数と説明変数の値を取得
target = train["Transported"].values
features_one = train[["Destination" , "Age"  , "HomePlanet" , "CryoSleep" , "VIP" , "RoomService" , "FoodCourt" , "ShoppingMall" , "Spa" , "VRDeck" , "CabinDeck" , "CabinNumber" , "CabinSide" , "ttl_spnd" , "Adult" , "PassengerGroup" , "PassengerNumber" , "PassengerGroupSize"]].values
 
# 決定木の作成
my_tree_one = tree.DecisionTreeClassifier()
my_tree_one = my_tree_one.fit(features_one, target)
 
# 「test」の説明変数の値を取得
test_features = test[["Destination" , "Age"  , "HomePlanet" , "CryoSleep" , "VIP" , "RoomService" , "FoodCourt" , "ShoppingMall" , "Spa" , "VRDeck" , "CabinDeck" , "CabinNumber" , "CabinSide" , "ttl_spnd" , "Adult" , "PassengerGroup" , "PassengerNumber" , "PassengerGroupSize"]].values
 
# 「test」の説明変数を使って「my_tree_one」のモデルで予測
my_prediction = my_tree_one.predict(test_features)

In [ ]:

# PassengerIdを取得
PassengerId = np.array(test["PassengerId"])
 
# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame(my_prediction, PassengerId, columns = ["Transported"])
 
# my_tree_one.csvとして書き出し
my_solution.to_csv("my_tree_one.csv", index_label = ["PassengerId"])

In [ ]:
seed=0
splittus=True
y=train['Transported']
y=y.astype(int)
#cats.remove('VIP')
print(train.columns)
if splittus:
    from sklearn.model_selection import train_test_split

    X=train.drop(columns=['Transported','PassengerId'])

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=seed,shuffle=True)
else:
    X_train=train.drop(columns=['Transported','PassengerId'])
    y_train=y

In [ ]:
from catboost import CatBoostClassifier
model=CatBoostClassifier(iterations=1000,#random_strength=0.1,cat_features=cats
                         eval_metric='Accuracy',
                        verbose=0)
if splittus:
    model.fit(X_train,y_train,eval_set=(X_val,y_val),use_best_model=True)
else:
    model.fit(X_train,y_train)
print(model.get_best_iteration())
print(model.random_seed_)
print(model.learning_rate_)

In [ ]:
#ランダムフォレストでやってみる⇒下がったのでいったん捨てる
#model=RandomForestClassifier(random_state=42)
#model.fit(X_train,y_train)

In [ ]:
test_cat=test.drop(columns=['PassengerId'])

ans=model.predict(test_cat)
ans=ans.astype(bool)
#ans.loc[ans['Transported']==0,'Transported']=False
#ans.loc[ans['Transported']==1,'Transported']=True

sub=pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
sub['Transported']=ans
sub.to_csv('./submission.csv',index=False)

In [ ]:
PassengerId